# Field - Circuit Coupled (Uncoupled?) Models of Machines and Transformers  
# Joint Course by Jianning and Domenico 

### Course Details 

1. Course title (to be defined?) and course code? 
2. 5 ECTS (as all other courses)
3. given in Q3 (?)
4. teaching by lectures and lab sessions
5. support in teaching non-teachnical skills by TBM 
6. not all students same trajectory through course - how to accomodate in teaching triangle?  
7. assessment by ... 

## Section 1: Introduction 

### Section 1.1: Concepts 

1. Coupled RLC <b>Circuit</b> models for current and voltages (and mechanical quantities) in and over windings. First order (non-)linear initial  value problems with sparse Jacobian;
2. Stationary and Quasi-stationary magnetic (and electric?) <b>Field</b> models 
3. Include <b>post-processing</b> for force, torque and inductance (not yet in EE4375)
4. Uncoupled solution: solve field first. Solve circuit subsequently.
5. Coupled solution: solve by plugging the field solve inside the circuit solve; 

### Section 2.1: Solution Methods

1. Circuit models (what of the following does Jianning desire to include): time-stepping: explicit vs. implicit (expect stiff problems to appear), fixed vs. variable time step, Runge-Kutta vs. Backward difference formulas, sparse linear system solve at each time step, assembly of the Jacobian at each time step, more, ...
2. Control models (what in required, how does it connect to other parts in the course?) 
3. Field models (what does Domenico have time for): 2D FEM on unstructured meshes and post-processing for the inductance;
4. Field - Circuit coupling; 


### Section 3.1: Implementations in Julia 
1. Jupyter Notebook System. Should we introduce Pluto instead? 
3. [sympy](sympy.org): symbolic reference solutions of linear initial (circuit) problems (include Laplace transforms?). Symbolic reference solutions of linear one-dimensional (only $x$) boundary (field) value problems;
4. [IJulia.jl](https://github.com/JuliaLang/IJulia.jl) 
5. [Plots.jl](https://docs.juliaplots.org/stable/): (does Jianning require alternative such as Makie.jl?)
6. [DifferentialEquations.jl](https://docs.sciml.ai/DiffEqDocs/stable/): numerical time integration of circuit problems. Numerical solution method (shooting method) for one-dimensional linear and non-linear boundary value problems;
7. [Gmsh.jl](https://github.com/JuliaFEM/Gmsh.jl): CAD modeling and Mesh generation
8. [Ferrite.jl](https://ferrite-fem.github.io/Ferrite.jl/stable/): finite element simulations; 

### Section 4.1: Examples 

1. coil-core-air configuration: define RLC circuit: see [notebook](https://github.com/ziolai/finite_element_electrical_engineering/blob/main/lab-sessions/2-lab-session-extension/2-lab-session-symbolic-solutions.ipynb) 
2. slotted permanent magnet machine: see [notebook](https://github.com/ziolai/finite_element_electrical_engineering/blob/main/project-based-assignment/metal-hydride-storage/notebooks/gmsh_tankNozzle.ipynb) for point of departure to define the geometry 
3. transformer (fault-current limiter)

## Section 2: Course Content 

### Block (1/3):Week 1 and 2 

1. Jianning: circuits. Current and voltages for interconnected coils. Initial value problem for coupled system of differential equations. Analytical solution methods? Time-stepping methods. Preriquisites to be specified. Reading material to be specified. Examples to be specified;
2. Domenico: fiels. Potential for electrical or magnetic fields. Boundary value problem for the Poisson equation. Post-processing for the inductance (and capacitance?). Analytical solution methods? Finite element methods. Preriquisites to be specified. Reading material to be specified. Example of coil-coil-air solved using sympy (analytical solution method - only linear material).
4. TMB: to be specified 

### Outdated: Preliminary Content Matter 

<b>Content</b>
1. Laplace and Poisson Equation for the Electrostatic Field in term of the electrical field potential. Material constitutive equations. Physical units;
2. electric energy density and magnetic energy density; 
3. (optional) Laplace and Poisson Equation for the Magnetostatic Field in terms of the scalar magnetic field potential. Material constitutive equations. Physical units;
4. double curl equation for the magnetic vector equation. Reduction to two-dimensional perpendicular current formulation;   
5. (optional): Reduction of double curl equation to to two-dimensional azimuthal current formulations; 
6. one-dimensional reference solutions: analytically, symbolically or built-in numerical tools;
7. one-dimensional coil-yoke-air problem: solve potential, post-process b-field, h-field, magnetic energy $W_m = \int_0^1 B_y(x) H_y(x) dx$, and compute inductance $L = 2 W_m/I^2$. Repeat with eddy currents, transient and magnetic saturation;  
8. two-dimensional reference solutions: analytically or built-in numerical tools;
9. Julia programminmg language   

<b>Provide Pointers to</b>
1. non-linear materials; 
2. eddy-current effects;
3. transient field computations; 

<b>Requires Input on</b>
1. material already available that can be reused (link here to courses in Q1 and Q2); 
2. how to compute inductance analytically or symbolically; 

### Block (2/3): Week 3 and 4

1. Jianning: state-space representation; 
2. Domenico: 2D FEM model for (1/2) two coils and core and (2/2) permanent magnet field in slotted machine. Poisson equation for magnetic vector potential component. Use implementation to explain concepts used.  Variational formulation. Galerkin spatial discretization. Element-by-element assembly. 
4. TBM: 

Assessment: 
1. derive the equations used in the model - derive the boundary conditions 
   
#### Outdated: One-Dimensional Finite Element Modeling 

<b>Content</b>
1. Problem formulation: ordinary differential equation supplied with boundary value problems;
2. Weak formulation, variational formulation, weighted residuals;
3. Mesh generation, finite element basis functions; 
4. Assembly mass matrix, stiffness matrix, load vector;
5. Linear system solve for finite element solution;
6. Post-processing; 

<b>Provide Pointers to</b>

<b>Requires Input on</b>
1. exact problem formulation;
2. details on computations of the inductance $L$ either using energy or flux and preferably both; 

### Block (3/3): (Week 5, 6 and 7)

1. Jianning: include mechanical variables in the output (observables) vector; 
2. Domenico: 2D FEM model for linear actuator 
3. (Poisson equation for magnetic vector potential component. Use implementation to explain concepts used.  Variational formulation. Galerkin spatial discretization. Element-by-element assembly.)
4. post-processing for forcec on the moving part; 
5. TBM: 

#### Outdated: Two-Dimensional Finite Element Modeling (adding air by union(air,air \setminus geometry) currently fials). 

<b>Content</b>
1. extend above from 1D (only $x$) to 2D (both $x$ and $y$); 

<b>Provide Pointers to</b>

<b>Requires Input on</b>
1. pre-cooking geometry definition and mesh generation;

## GMSH Manual 

<b>gmsh.model.occ.add_curve_loop</b>: add closed loops around circle. See e.g. L21 of [tq19.jl](https://gitlab.onelab.info/gmsh/gmsh/blob/gmsh_4_15_0/tutorials/julia/t19.jl#L21). Does <b>not</b> work for rectangles (because surface instead of loop). 

<b>gmsh.model.occ.add_surface_loop</b>: add closed loops around surface. Not yet seen in tutorials. Does not work for circle (because loop instead of surface). 

<b>gmsh.model.occ.add_surface_loop</b>: See L30 of [t16.jl](https://gitlab.onelab.info/gmsh/gmsh/blob/gmsh_4_15_0/tutorials/julia/t16.jl#L30). What happens in case the two seperate objects are formed? 

In [ ]:
?gmsh.model.occ.add_curve_loop

In [ ]:
?gmsh.model.occ.add_surface_loop

In [71]:
?gmsh.model.occ.cut

```
gmsh.model.occ.cut(objectDimTags, toolDimTags, tag = -1, removeObject = true, removeTool = true)
```

Compute the boolean difference between the entities `objectDimTags` and `toolDimTags` (given as vectors of (dim, tag) pairs) in the OpenCASCADE CAD representation. Return the resulting entities in `outDimTags`. If `tag` is positive, try to set the tag explicitly (only valid if the boolean operation results in a single entity). Remove the object if `removeObject` is set. Remove the tool if `removeTool` is set.

Return `outDimTags`, `outDimTagsMap`.

Types:

  * `objectDimTags`: vector of pairs of integers
  * `toolDimTags`: vector of pairs of integers
  * `outDimTags`: vector of pairs of integers
  * `outDimTagsMap`: vector of vectors of pairs of integers
  * `tag`: integer
  * `removeObject`: boolean
  * `removeTool`: boolean


## Examples from Ferrite.jl   

Ferrite tutorial as is. 

In [52]:
# Ferrite NS tutorial 

#..initialize GMSH 
gmsh.initialize()

gmsh.option.set_number("General.Verbosity", 2)
dim = 2;
zc = 0 # z-coordinate 

#..outer rectangle
rect_tag = gmsh.model.occ.add_rectangle(0, 0, 0, 1.1, 0.41)
#..inner circle 
circle_tag = gmsh.model.occ.add_circle(0.2, 0.2, 0, 0.05)
#..disk enclosured by circle 
circle_curve_tag = gmsh.model.occ.add_curve_loop([circle_tag])
circle_surf_tag = gmsh.model.occ.add_plane_surface([circle_curve_tag])
gmsh.model.occ.cut([(dim, rect_tag)], [(dim, circle_surf_tag)])

gmsh.model.occ.synchronize()

# gmsh.option.setNumber("Mesh.MeshSizeMax",0.001)

gmsh.model.mesh.generate(dim)
# grid = togrid()

#..if true, write mesh to file for further processing
#if (true) gmsh.write("ferrite-tut.geom") end
if (true) gmsh.write("ferrite-tut.msh") end 

#..if true, visualize mesh through the GUI 
if (true) gmsh.fltk.run() end 

#..finalize GMSH
gmsh.finalize()

Same as before, replacing circle by disk. 

In [53]:
# Modified Ferrite NS tutorial: replace circle by disk 

#..initialize GMSH 
gmsh.initialize()

gmsh.option.set_number("General.Verbosity", 2)
dim = 2;
zc = 0 # z-coordinate 

#..outer rectangle
rect_tag = gmsh.model.occ.add_rectangle(0, 0, 0, 1.1, 0.41)
#..inner disk 
disk_tag = gmsh.model.occ.add_disk(0.2, 0.2, 0, 0.05, 0.05)
#..difference between rectangle and disk  
gmsh.model.occ.cut([(dim, rect_tag)], [(dim, disk_tag)])

gmsh.model.occ.synchronize()

# gmsh.option.setNumber("Mesh.MeshSizeMax",0.001)

gmsh.model.mesh.generate(dim)
# grid = togrid()

#..if true, write mesh to file for further processing
#if (true) gmsh.write("ferrite-tut.geom") end
if (true) gmsh.write("ferrite-tut.msh") end 

#..if true, visualize mesh through the GUI 
if (true) gmsh.fltk.run() end 

#..finalize GMSH
gmsh.finalize()

Same as before, replacing disk by rectangle. 

In [55]:
#..initialize GMSH 
gmsh.initialize()

gmsh.option.set_number("General.Verbosity", 2)
dim = 2;
zc = 0 # z-coordinate 

#..outer rectangle
rect_tag = gmsh.model.occ.add_rectangle(0, 0, 0, 1.1, 0.41)
#..inner disk 
inn_rect_tag = gmsh.model.occ.add_rectangle(0.2, 0.2, 0, 0.05, 0.05)
#..difference between rectangle and disk  
gmsh.model.occ.cut([(dim, rect_tag)], [(dim, inn_rect_tag)])

gmsh.model.occ.synchronize()

# gmsh.option.setNumber("Mesh.MeshSizeMax",0.001)

gmsh.model.mesh.generate(dim)
# grid = togrid()

#..if true, write mesh to file for further processing
#if (true) gmsh.write("ferrite-tut.geom") end
if (true) gmsh.write("ferrite-tut.msh") end 

#..if true, visualize mesh through the GUI 
if (true) gmsh.fltk.run() end 

#..finalize GMSH
gmsh.finalize()

## Actuator Geometry 

In [9]:
try
    using Gmsh: gmsh
catch
    using gmsh
end 

In [46]:
gmsh.finalize();

Same as before, replacing circle by rectangle. 

In [25]:
?gmsh.model.occ.add_curve_loop

```
gmsh.model.occ.addCurveLoop(curveTags, tag = -1)
```

Add a curve loop (a closed wire) in the OpenCASCADE CAD representation, formed by the curves `curveTags`. `curveTags` should contain tags of curves forming a closed loop. Negative tags can be specified for compatibility with the built-in kernel, but are simply ignored: the wire is oriented according to the orientation of its first curve. Note that an OpenCASCADE curve loop can be made of curves that share geometrically identical (but topologically different) points. If `tag` is positive, set the tag explicitly; otherwise a new tag is selected automatically. Return the tag of the curve loop.

Return an integer.

Types:

  * `curveTags`: vector of integers
  * `tag`: integer


In [45]:
# Ferrite NS tutorial 

#..initialize GMSH 
gmsh.initialize()

gmsh.option.set_number("General.Verbosity", 2)
dim = 2;
zc = 0 # z-coordinate 

#..outer rectangle 
rect_tag = gmsh.model.occ.add_rectangle(0, 0, 0, 1.1, 0.41)
rect_curve_tag = gmsh.model.occ.add_curve_loop([rect_tag])
#..inner rectangle 
#inn_rect_tag = gmsh.model.occ.add_rectangle(0.2, 0.2, 0, 0.05, 0.05)
#..curved loop surrounding the inner rectangle
#inn_rect_curve_tag = gmsh.model.occ.add_curve_loop([inn_rect_tag])
#inn_rect_surf_tag = gmsh.model.occ.add_plane_surface([inn_rect_curve_tag])
#gmsh.model.occ.cut([(dim, rect_tag)], [(dim, inn_rect_surf_tag)])

gmsh.model.occ.synchronize()

# gmsh.option.setNumber("Mesh.MeshSizeMax",0.001)

gmsh.model.mesh.generate(dim)
# grid = togrid()

#..if true, write mesh to file for further processing
#if (true) gmsh.write("actuator.geom") end
if (true) gmsh.write("ferrite-tut.msh") end 

#..if true, visualize mesh through the GUI 
if (true) gmsh.fltk.run() end 

#..finalize GMSH
gmsh.finalize()

Error   : Curve loop is not closed


LoadError: Curve loop is not closed

In [87]:
gmsh.finalize();

Error   : Gmsh has not been initialized
Error   : Gmsh has not been initialized


LoadError: UndefVarError: `error` not defined

In [89]:
# does the operation cut require information on the boundaries? 
#..set geometrical constants.. 
L = 1. 
R = 0.1
Lin = 0.1*L 
Rin = 0.3*R 

#..initialize GMSH 
gmsh.initialize()

gmsh.option.set_number("General.Verbosity", 2)
dim = 2;
zc = 0 # z-coordinate 
#..core.. 
out_tag       = gmsh.model.occ.add_rectangle(-0.05, 0, zc, 0.1, 0.0405)
#inn_tag       = gmsh.model.occ.add_rectangle(-0.04, 0.01, zc, .08, .0205)
inn_tag       = gmsh.model.occ.add_rectangle(-0.04, 0.01, zc, .1, .0205)
core_tag      = gmsh.model.occ.cut([(2, out_tag)],  [(2, inn_tag)])

#gap_tag  = gmsh.model.occ.add_rectangle(-0.021, 0.0305, zc, .042, .01)
#core_tag = gmsh.model.occ.cut([(2, out_tag)],  [(2, inn_tag)])
#core_tag = gmsh.model.occ.cut([(2, out_tag)],  [(2, gap_tag)])
#core_curve_tag = gmsh.model.occ.add_curve_loop([core_tag])
#..air 
air_tag = gmsh.model.occ.add_rectangle(-0.08, -0.03, zc, 0.16, 0.09)
air_tag = gmsh.model.occ.cut([(2, air_tag)],  [(2, core_tag)]) 
#air2_tag = gmsh.model.occ.cut([(2, air_tag)],  [(2, core_tag)]) 
#air_tag = gmsh.model.occ.cut([(2, air_tag)],  [(2, mover_tag)])
#air_tag = gmsh.model.occ.cut([(2, air_tag)],  [(2, pm_tag)])

gmsh.model.occ.synchronize()

#..4/7: assign physical groups
#gmsh.model.addPhysicalGroup(1, [l1, l2, l3, l4, l5, l7, l8], -1, "wall")
#gmsh.model.addPhysicalGroup(1, [l6], -1, "inlet")
#gmsh.model.addPhysicalGroup(2, [surf], -1, "omega")

gmsh.option.setNumber("Mesh.MeshSizeMax",0.001)

gmsh.model.mesh.generate(dim)
# grid = togrid()

#..if true, write mesh to file for further processing
#if (true) gmsh.write("actuator.geom") end
if (true) gmsh.write("actuator.msh") end 

#..if true, visualize mesh through the GUI 
if (true) gmsh.fltk.run() end 

#..finalize GMSH
gmsh.finalize()

LoadError: MethodError: [0mCannot `convert` an object of type [92mTuple{Vector{Tuple{Int32, Int32}}, Vector{Vector{Tuple{Int32, Int32}}}}[39m[0m to an object of type [91mInt32[39m

[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::T[39m) where T<:Number
[0m[90m   @[39m [90mBase[39m [90m[4mnumber.jl:6[24m[39m
[0m  convert(::Type{T}, [91m::T[39m) where T
[0m[90m   @[39m [90mBase[39m [90m[4mBase.jl:84[24m[39m
[0m  convert(::Type{T}, [91m::Number[39m) where T<:Number
[0m[90m   @[39m [90mBase[39m [90m[4mnumber.jl:7[24m[39m
[0m  ...


In [86]:
surf_inn_tag

LoadError: UndefVarError: `surf_inn_tag` not defined

In [15]:
#..set geometrical constants.. 
L = 1. 
R = 0.1
Lin = 0.1*L 
Rin = 0.3*R 

#..initialize GMSH 
gmsh.initialize()

gmsh.option.set_number("General.Verbosity", 2)
dim = 2;
zc = 0 # z-coordinate 
#..core.. 
out_tag  = gmsh.model.occ.add_rectangle(-0.05, 0, zc, 0.1, 0.0405)
inn_tag  = gmsh.model.occ.add_rectangle(-0.04, 0.01, zc, .08, .0205)
gap_tag  = gmsh.model.occ.add_rectangle(-0.021, 0.0305, zc, .042, .01)
core_tag = gmsh.model.occ.cut([(2, out_tag)],  [(2, inn_tag)])
core_tag = gmsh.model.occ.cut([(2, out_tag)],  [(2, gap_tag)])
#..mover.. 
mover_tag = gmsh.model.occ.add_rectangle(-0.020, 0.0305, zc, .04, .01)
#..permanent magnet.. 
pm_tag    = gmsh.model.occ.add_rectangle(-0.010, 0.01, zc, .02, .02)
#..left coil (consisting of an up and down part).. 
left_coil_up  = gmsh.model.occ.add_rectangle(-0.04, 0.01, zc, 0.03, 0.01)
left_coil_dw  = gmsh.model.occ.copy([(2, left_coil_up)])
left_coil_dw  = gmsh.model.occ.translate(left_coil_dw, 0, -0.02, 0)
#..right coil (consisting of an up and down part).. 
right_coil_up = gmsh.model.occ.add_rectangle(0.01, 0.01, zc, 0.03, 0.01)
right_coil_dw = gmsh.model.occ.copy([(2, right_coil_up)])
right_coil_dw = gmsh.model.occ.translate(right_coil_dw, 0, -0.02, 0)
#..air 
air_tag = gmsh.model.occ.add_rectangle(-0.08, -0.03, zc, 0.16, 0.09)
air_tag = gmsh.model.occ.cut([(2, air_tag)],  [(2, core_tag)])
#air_tag = gmsh.model.occ.cut([(2, air_tag)],  [(2, mover_tag)])
#air_tag = gmsh.model.occ.cut([(2, air_tag)],  [(2, pm_tag)])
gmsh.model.occ.synchronize()

#..4/7: assign physical groups
#gmsh.model.addPhysicalGroup(1, [l1, l2, l3, l4, l5, l7, l8], -1, "wall")
#gmsh.model.addPhysicalGroup(1, [l6], -1, "inlet")
#gmsh.model.addPhysicalGroup(2, [surf], -1, "omega")

gmsh.option.setNumber("Mesh.MeshSizeMax",0.001)

gmsh.model.mesh.generate(dim)
# grid = togrid()

#..if true, write mesh to file for further processing
#if (true) gmsh.write("actuator.geom") end
if (true) gmsh.write("actuator.msh") end 

#..if true, visualize mesh through the GUI 
if (true) gmsh.fltk.run() end 

#..finalize GMSH
gmsh.finalize()

LoadError: MethodError: [0mCannot `convert` an object of type [92mTuple{Vector{Tuple{Int32, Int32}}, Vector{Vector{Tuple{Int32, Int32}}}}[39m[0m to an object of type [91mInt32[39m

[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::T[39m) where T<:Number
[0m[90m   @[39m [90mBase[39m [90m[4mnumber.jl:6[24m[39m
[0m  convert(::Type{T}, [91m::T[39m) where T
[0m[90m   @[39m [90mBase[39m [90m[4mBase.jl:84[24m[39m
[0m  convert(::Type{T}, [91m::Number[39m) where T<:Number
[0m[90m   @[39m [90mBase[39m [90m[4mnumber.jl:7[24m[39m
[0m  ...


In [17]:
gmsh.finalize();

In [19]:
#..set geometrical constants.. 
L = 1. 
R = 0.1
Lin = 0.1*L 
Rin = 0.3*R 

#..initialize GMSH 
gmsh.initialize()

gmsh.option.set_number("General.Verbosity", 2)
dim = 2;
zc = 0 # z-coordinate 
#..core.. 
out_tag  = gmsh.model.occ.add_rectangle(-0.05, 0, zc, 0.1, 0.0405)
inn_tag  = gmsh.model.occ.add_rectangle(-0.04, 0.01, zc, .08, .0205)
gap_tag  = gmsh.model.occ.add_rectangle(-0.021, 0.0305, zc, .042, .01)
core_tag = gmsh.model.occ.cut([(2, out_tag)],  [(2, inn_tag)])
core_tag = gmsh.model.occ.cut([(2, out_tag)],  [(2, gap_tag)])
#..mover.. 
mover_tag = gmsh.model.occ.add_rectangle(-0.020, 0.0305, zc, .04, .01)
#..permanent magnet.. 
pm_tag    = gmsh.model.occ.add_rectangle(-0.010, 0.01, zc, .02, .02)
#..left coil (consisting of an up and down part).. 
left_coil_up  = gmsh.model.occ.add_rectangle(-0.04, 0.01, zc, 0.03, 0.01)
left_coil_dw  = gmsh.model.occ.copy([(2, left_coil_up)])
left_coil_dw  = gmsh.model.occ.translate(left_coil_dw, 0, -0.02, 0)
#..right coil (consisting of an up and down part).. 
right_coil_up = gmsh.model.occ.add_rectangle(0.01, 0.01, zc, 0.03, 0.01)
right_coil_dw = gmsh.model.occ.copy([(2, right_coil_up)])
right_coil_dw = gmsh.model.occ.translate(right_coil_dw, 0, -0.02, 0)
#..air 
#air_tag = gmsh.model.occ.add_rectangle(-0.08, -0.03, zc, 0.16, 0.09)
#air_tag = gmsh.model.occ.cut([(2, air_tag)],  [(2, core_tag)])
#air_tag = gmsh.model.occ.cut([(2, air_tag)],  [(2, mover_tag)])
#air_tag = gmsh.model.occ.cut([(2, air_tag)],  [(2, pm_tag)])
gmsh.model.occ.synchronize()

#..4/7: assign physical groups
#gmsh.model.addPhysicalGroup(1, [l1, l2, l3, l4, l5, l7, l8], -1, "wall")
#gmsh.model.addPhysicalGroup(1, [l6], -1, "inlet")
#gmsh.model.addPhysicalGroup(2, [surf], -1, "omega")

gmsh.option.setNumber("Mesh.MeshSizeMax",0.001)

gmsh.model.mesh.generate(dim)
# grid = togrid()

#..if true, write mesh to file for further processing
#if (true) gmsh.write("actuator.geom") end
if (true) gmsh.write("actuator.msh") end 

#..if true, visualize mesh through the GUI 
if (true) gmsh.fltk.run() end 

#..finalize GMSH
gmsh.finalize()